In [ ]:
# 获取价格
# 获取token的address
token_list = ['ETH',"WBTC","USDC.e"]
address_list = [x for x in symbol.index if symbol.loc[x,'data.symbol'] in token_list]
prc = {}

for address in address_list:
  query = f'''query {{
    fastPrices(token: "{address}", first: 1000, orderBy: id, orderDirection: desc, timestamp_gt: 0, timestamp_lt: 1801001000, where: {{period: "hourly"}}) {{
      id
      token
      value
      period
      timestamp
    }}
  }}'''
  url = 'https://subgraph.satsuma-prod.com/3b2ced13c8d9/gmx/gmx-arbitrum-stats/api'
  headers = {'Content-Type': 'application/json'}
  request_json = {'query': query}
  response = requests.post(url, headers=headers, data=json.dumps(request_json))
  price_data = response.json()
  if 'errors' in price_data:
      error_message = price_data['errors'][0]['message']
      raise Exception(f"GraphQL query failed: {error_message}")
  price_data = price_data['data']['fastPrices']
  price_data = pd.DataFrame(price_data)
  data['time'] = pd.to_datetime(price_data['timestamp'], unit='s')
  # 生成token对应的名字
  price_data['token'] = price_data['token'].apply(lambda x: symbol.loc[x]['data.symbol'])
  price_data['value'] = price_data['value'].astype(float) / 10**30
  token_name = symbol.loc[address]['data.symbol']
  prc[token_name] = price_data